<a href="https://colab.research.google.com/github/greymouse1/spatialanalysis/blob/main/continuity_multiple_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code is licenced under MIT licence.

Author: Nikola G.

Credits:

Tutorial from Momepy package website at http://docs.momepy.org/en/stable/user_guide/graph/coins.html
based on paper by Tripathy et al. (2020)

OpenAI. (2024). ChatGPT (version 4) [Large language model]. OpenAI. https://openai.com/chatgpt



Tripathy, P., Rao, P., Balakrishnan, K., & Malladi, T. (2020). An open-source tool to extract natural continuity and hierarchy of urban street networks. Environment and Planning B: Urban Analytics and City Science. http://dx.doi.org/10.1177/2399808320967680

In [1]:
!pip install osmnx > /dev/null 2>&1
!pip install momepy > /dev/null 2>&1
!pip install mapclassify>=2.4.0 > /dev/null 2>&1 # install mapclassify with version >=2.4.0
!pip install powerlaw > /dev/null 2>&1
import osmnx as ox
import geopandas as gpd
import momepy
import mapclassify
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import powerlaw
from collections import defaultdict, Counter
from shapely.geometry import MultiLineString, LineString, Point

In [7]:
def naturalCities(currentCity,folderPath,cityName):
  # Retrieve the graph within the polygon's boundaries
  # This will pull OSM data from inside the polygon and create a networkX graph

  graph = ox.graph_from_polygon(
      currentCity,
      network_type='drive',  # Choose network type (e.g., 'drive', 'walk', 'bike', etc.)
      simplify=True,         # Simplify graph (remove unnecessary nodes)
      retain_all=False,      # Keep only the largest connected component
      truncate_by_edge=False  # Truncate by edge to keep nodes near the edge
  )

  # Reproject graph
  # Choice of final projection is automatic, original must be WGS84

  city_streets = ox.projection.project_graph(graph)

  # Create gdf from graph so it can be used later on

  city_gdf = ox.graph_to_gdfs(
      ox.convert.to_undirected(city_streets),
      nodes=False,
      edges=True,
      node_geometry=False,
      fill_edge_geometry=True,
  )

  # Calculate continuity from the gdf

  continuity = momepy.COINS(city_gdf, angle_threshold=135, flow_mode=False)

  # Pull out stroke

  city_stroke_gdf = continuity.stroke_gdf()

  # Save stroke to .shp
  shapefile_path = os.path.join(folderPath, f"polygon_{cityName}.shp")
  city_stroke_gdf.to_file(shapefile_path)
#--------------------------------------------------------------------------------------------------------------
  # Initialize the vertex-to-linestring mapping
  vertex_to_linestring = defaultdict(list)

  # Iterate over geometries and map vertices to LineStrings (handle MultiLineString)
  for idx, geom in city_stroke_gdf.geometry.items():
      if isinstance(geom, LineString):  # Process single LineString
          for point in geom.coords:
              vertex_to_linestring[Point(point)].append(idx)
      elif isinstance(geom, MultiLineString):  # Process MultiLineString
          # For each LineString in MultiLineString
          for subline in geom.geoms:
              for point in subline.coords:
                  vertex_to_linestring[Point(point)].append(idx)

  # Initialize a dictionary to store the connection counts for each line
  line_connections = defaultdict(int)

  # Iterate over the geometries again to count connections
  for idx, geom in city_stroke_gdf.geometry.items():
      if isinstance(geom, LineString):  # Process single LineString
          connections = []  # Use a list to count multiple connections
          for point in geom.coords:
              for connected_line in vertex_to_linestring[Point(point)]:
                  if connected_line != idx:
                      connections.append(connected_line)  # Add connection to the list
          line_connections[idx] = len(connections)  # Store the total number of connections
      elif isinstance(geom, MultiLineString):  # Process MultiLineString
          # For each LineString in MultiLineString
          for subline in geom.geoms:
              connections = []  # Use a list to count multiple connections for each subline
              for point in subline.coords:
                  for connected_line in vertex_to_linestring[Point(point)]:
                      if connected_line != idx:
                          connections.append(connected_line)  # Add connection to the list
              # Store the connection count for each subline (if necessary)
              # For now, we store the count for the entire MultiLineString
              line_connections[idx] = len(connections)


  # Extract degree (number of connections) values
  degree_values = list(line_connections.values())

  # Count frequencies of degree values
  degree_counts = Counter(degree_values)

  # Extract x (degrees) and y (frequencies)
  x = np.array(list(degree_counts.keys()))       # Unique degree values
  y = np.array(list(degree_counts.values()))    # Frequency of each degree

  # Fit the degree distribution to a power-law model
  fit = powerlaw.Fit(degree_values)
  print(f"Alpha: {fit.alpha}")
  print(f"xmin: {fit.xmin}")

  # Get alpha and xmin (scaling parameter and lower bound for the power-law fit)
  alpha = fit.alpha
  xmin = fit.xmin

  # Get the p-value from the goodness-of-fit test
  p_value = fit.power_law.KS()

  print(f"p-value: {p_value}")
  print("-----------------------")
  # Compare the power-law fit with an alternative distribution (e.g., exponential)
  R, p_alt = fit.distribution_compare('power_law', 'exponential')
  print(f"Log-likelihood ratio (R): {R}")
  print(f"p-value for comparison: {p_alt}")

  # Plot the data and the fitted power law
  fig = fit.plot_pdf(marker='o', color='blue', markersize=4, label='Empirical Data')
  fit.power_law.plot_pdf(ax=fig, color='red', linestyle='--', linewidth=1, label='Power Law Fit')

  # Add legend and labels
  plt.xlabel("Degree (Connections)")
  plt.ylabel("Frequency")
  plt.legend()

  output_path_power = os.path.join(folderPath, f"power_{cityName}.png")
  plt.savefig(output_path_power, dpi=600, bbox_inches="tight")

  print(f"Plot for power fit for {cityName} saved to {output_path_power}")
  plt.clf() # clear existing plot and make space for a new one

#-------------------------
  # Add connections to original gdf
  city_stroke_gdf['n_connections'] = degree_values

  # Calculate heavy tailed classification
  classifier = mapclassify.HeadTailBreaks(degree_values)

  # Get classification details
  class_intervals = classifier.bins  # Class boundaries
  counts = classifier.counts         # Number of features in each class
  labels = classifier.yb             # Class labels for each feature

  # Save classification output to a text file
  classifier_path = os.path.join(folderPath, f"map_classifier_output_{cityName}.txt")

  with open(classifier_path, "w") as file:
      file.write("HeadTailBreaks Classification\n")
      file.write("================================\n")
      file.write("Total segments:\n")
      file.write(f"{len(degree_values)}" + "\n\n")
      file.write("HT index of street connectivity:\n")
      file.write(f"{len(class_intervals)}" + "\n\n")
      file.write("Class Intervals:\n")
      file.write(", ".join(f"{b:.2f}" for b in class_intervals) + "\n\n")
      file.write("Counts in Each Class:\n")
      file.write(", ".join(str(c) for c in counts) + "\n\n")
      file.write("Statistical indicators for power law:" + "\n\n")
      file.write("Alpha: " + str(alpha) + "\n")
      file.write("xmin: " + str(xmin) + "\n")
      file.write("p-value: " + str(p_value) + "\n")
      file.write("Log-likelihood ratio (R): " + str(R) + "\n")
      file.write("p-value for comparison: " + str(p_alt) + "\n")


  print(f"Classifier output saved to {classifier_path}")

  # Show the classifier (lower and upper bounds plus count)
  print(classifier)

  fig, ax = plt.subplots(figsize=(15, 15))

  city_stroke_gdf.plot(
      cmap="jet",
      column="n_connections",
      legend=True,
      scheme="headtailbreaks",
      linewidth=0.5,
      ax=ax,
  ).set_axis_off()

  fig.patch.set_facecolor("black")

  output_path_figure = os.path.join(folderPath, f"figure_{cityName}.png")
  plt.savefig(output_path_figure, dpi=600, bbox_inches="tight")
  plt.close(fig)
  print(f"Plot for {cityName} saved to {output_path_figure}")

  # Histogram for n_segments

  city_stroke_gdf['n_connections'].plot(kind='hist', bins=40, title='n_segments')
  plt.gca().spines[['top', 'right',]].set_visible(False)
  output_path_histogram = os.path.join(folderPath, f"histogram_{cityName}.png")
  plt.savefig(output_path_histogram, dpi=300, bbox_inches="tight")
  print(f"Histogram for {cityName} saved to {output_path_histogram}")
  plt.clf() # clear existing plot and make space for a new one

In [8]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load shp file with all the polygons
all_pol = gpd.read_file("/content/drive/MyDrive/spatialanalysis/all_pol_names/all_pol_names.shp")

base_folder = "output_polygons"
os.makedirs(base_folder, exist_ok=True)  # Create base folder if it doesn't exist

# Iterate through each polygon and create a new GeoDataFrame
for index, row in all_pol.iterrows():

    # Create a new GeoDataFrame for the current polygon
    new_gdf = gpd.GeoDataFrame(
        [row],
        columns=all_pol.columns,
        crs=all_pol.crs  # Retain the original CRS
    )
    polygon = new_gdf.geometry.iloc[0]  # Extract the Polygon/MultiPolygon geometry
    city_name = row['name']  # Extract the city name from the row

    # Define the folder name for the current polygon
    polygon_folder = os.path.join(base_folder, f"{city_name}")

    # Save or process the new GeoDataFrame
    print(f"New GeoDataFrame for {city_name} is loaded")
    if city_name == "Java":
      next
    else:
      os.makedirs(polygon_folder, exist_ok=True)  # Create folder if it doesn't exist
      naturalCities(polygon,polygon_folder,city_name)





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
New GeoDataFrame for Surabaya is loaded
New GeoDataFrame for Malang is loaded
New GeoDataFrame for Surakarta is loaded
New GeoDataFrame for Yogyakarta is loaded
New GeoDataFrame for Semarang is loaded


<ipython-input-7-cf3f053cd33b>:38: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  city_stroke_gdf.to_file(shapefile_path)
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'stroke_group' to 'stroke_gro'
  ogr_write(
Values less than or equal to 0 in data. Throwing out 0 or negative values


Calculating best minimal value for power law fit
Alpha: 3.517338325295844
xmin: 9.0
p-value: 0.03906426263530771
-----------------------
Log-likelihood ratio (R): 201.56035188227668
p-value for comparison: 3.3390667260192387e-12
Plot for power fit for Semarang saved to output_polygons/Semarang/power_Semarang.png
Classifier output saved to output_polygons/Semarang/map_classifier_output_Semarang.txt
HeadTailBreaks

    Interval       Count
------------------------
[  0.00,   3.39] | 16359
(  3.39,   7.35] |  4511
(  7.35,  13.54] |  1254
( 13.54,  23.52] |   343
( 23.52,  38.93] |    96
( 38.93,  59.77] |    34
( 59.77,  94.31] |     9
( 94.31, 129.00] |     3
(129.00, 163.00] |     1
Plot for Semarang saved to output_polygons/Semarang/figure_Semarang.png
Histogram for Semarang saved to output_polygons/Semarang/histogram_Semarang.png
New GeoDataFrame for Bandung is loaded
New GeoDataFrame for Jakarta is loaded


<Figure size 640x480 with 0 Axes>

In [4]:
# Zip files for download

!zip -r /content/output_polygons.zip /content/output_polygons

  adding: content/output_polygons/ (stored 0%)
  adding: content/output_polygons/Semarang/ (stored 0%)
  adding: content/output_polygons/Semarang/polygon_Semarang.shx (deflated 53%)
  adding: content/output_polygons/Semarang/histogram_Semarang.png (deflated 27%)
  adding: content/output_polygons/Semarang/polygon_Semarang.cpg (stored 0%)
  adding: content/output_polygons/Semarang/map_classifier_output_Semarang.txt (deflated 32%)
  adding: content/output_polygons/Semarang/power_Semarang.png (deflated 20%)
  adding: content/output_polygons/Semarang/polygon_Semarang.dbf (deflated 90%)
  adding: content/output_polygons/Semarang/polygon_Semarang.shp (deflated 41%)
  adding: content/output_polygons/Semarang/figure_Semarang.png (deflated 2%)
  adding: content/output_polygons/Semarang/polygon_Semarang.prj (deflated 37%)
